## Load data

In [1]:
import librosa as lb
import pandas as pd
import json
import os
import csv
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
audio_dir = '/home/kvu/erc/data/AFEW/raw-audios'
json_path = '/home/kvu/erc/data/AFEW/labels.json'

train_dir = audio_dir + '/train'
val_dir = audio_dir + '/val'
test_dir = audio_dir + '/test'

In [3]:
with open(json_path) as f:
    dataset_json = json.load(f)

In [4]:
def aggregate_metadata(phase, audio_dir):
    labels = dataset_json[phase]
    df = pd.DataFrame(columns=['audio_id', 'label', 'duration', 'sampling_rate'])
    cur_row = 0
    
    for audio_name in tqdm(os.listdir(audio_dir)):
        audio_id = audio_name[:-4]
        label = labels[audio_id]
        y, sr = lb.load(os.path.join(audio_dir, audio_name))
        dur = lb.get_duration(y, sr)
        df.loc[cur_row] = [audio_id, label, dur, sr]
        cur_row += 1
    
    return df

In [5]:
train_df = aggregate_metadata('train', train_dir)

100%|██████████| 773/773 [03:02<00:00,  4.24it/s]


In [6]:
train_df

,audio_id,label,duration,sampling_rate
0,004235880,sad,4.008027,22050
1,004850694,neutral,3.408027,22050
2,003708960,sad,3.408027,22050
3,004212680,happy,1.248027,22050
4,010730723,sad,4.680000,22050
...,...,...,...,...
768,003345000,angry,3.048027,22050
769,004601560,surprise,2.040000,22050
770,002014840,angry,3.408027,22050
771,005737640,angry,4.200000,22050


## EDA

In [7]:
from pandas_profiling import ProfileReport

In [8]:
report = ProfileReport(train_df)

In [9]:
report

Summarize dataset:   0%|          | 0/17 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
report.to_file("afew_EDA.html")

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Get CSVs

In [10]:
classes_filter = ['happy', 'neutral', 'angry', 'sad', 'fear', 'surprise', 'disgust']
classes_mapping = {'happy': 0, 'neutral': 1, 'angry': 2, 'sad': 3, 'fear': 4, 'surprise': 5, 'disgust': 6}

In [11]:
def json2csv(phase, audio_dir, csv_path):
    row_list = []
    row_list.append(['audio_id', 'label'])
    labels = dataset_json[phase]
    for audio_name in tqdm(os.listdir(audio_dir)):
        audio_id = audio_name[:-4]
        label = labels[audio_id]
        if label in classes_filter:
            row_list.append([audio_id, classes_mapping[label]])
            
    with open(csv_path, 'w', newline='') as f:
        writer = csv.writer(f, delimiter=',')
        writer.writerows(row_list)

In [12]:
json2csv('train', train_dir, '/home/kvu/erc/data/AFEW/raw-audios/train.csv')

100%|██████████| 773/773 [00:00<00:00, 834112.94it/s]


In [13]:
json2csv('val', val_dir, '/home/kvu/erc/data/AFEW/raw-audios/val.csv')

100%|██████████| 383/383 [00:00<00:00, 287579.38it/s]
